In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers.embeddings import Embedding
from keras.models import Model
from keras.layers import Concatenate, Dense, Input, concatenate,Flatten, Reshape
import keras

tf.compat.v1.enable_eager_execution()

Using TensorFlow backend.


In [2]:
dataset=pd.read_csv("avazuTrainDataset.csv", na_values = "?")

In [3]:
dataset.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,10006777279679619273,0,2014-10-21 00:00:00,1005,0,85f751fd,c4e18dd6,50e219e0,d2bb6502,2347f47a,...,1,3,18987,320,50,2158,3,291,100193,61
1,10017726656797105592,0,2014-10-21 00:00:00,1005,0,543a539e,c7ca3108,3e814130,ecad2386,7801e8d9,...,1,0,20352,320,50,2333,0,39,-1,157
2,10017963219731982376,0,2014-10-21 00:00:00,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,-1,79
3,10020717995127086125,0,2014-10-21 00:00:00,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,-1,79
4,10022961149355219548,0,2014-10-21 00:00:00,1005,0,85f751fd,c4e18dd6,50e219e0,e2fcccd2,5c5a694b,...,1,0,4687,320,50,423,2,39,100148,32


In [4]:
#The dasaset has an imbalance on label.Therefore we will make it balanced randomly choosing more data with respect to click==1
dataset["click"].value_counts()

0    830536
1    169464
Name: click, dtype: int64

In [5]:
dataset=pd.read_csv("balancedAvazuTrainDataset.csv", na_values = "?")

In [6]:
#we have chose our data balanced which is "# of clicked=0 equals to # of clicked=1"
dataset["click"].value_counts()

1    500000
0    500000
Name: click, dtype: int64

In [7]:
#Check for NaN values
dataset.isna().sum()

Unnamed: 0          0
id                  0
click               0
hour                0
C1                  0
banner_pos          0
site_id             0
site_domain         0
site_category       0
app_id              0
app_domain          0
app_category        0
device_id           0
device_ip           0
device_model        0
device_type         0
device_conn_type    0
C14                 0
C15                 0
C16                 0
C17                 0
C18                 0
C19                 0
C20                 0
C21                 0
dtype: int64

In [8]:
#Selecting user_id and item_id
questionDf=pd.DataFrame(columns=["user_id","item_id","target"])
questionDf["user_id"]=dataset["id"].copy()
questionDf["item_id"]=dataset["app_id"].copy()
questionDf["target"]=dataset["click"].copy()
#Training was so slow for my machine with 1M samples,therefore I reduced the sample size (sorry!)
questionDf=questionDf.sample(n=500000)
questionDf.reset_index(drop=True,inplace=True)
questionDf

,user_id,item_id,target
0,13608768438520409857,ecad2386,0
1,14692274859831145929,92f5800b,0
2,17090449056914370228,ecad2386,0
3,7188639805966656812,ecad2386,0
4,245941207644775581,ecad2386,1
...,...,...,...
499995,377358469892694884,7358e05e,1
499996,14653219386271282421,92f5800b,0
499997,13894820425956565476,ecad2386,0
499998,15207322995760605924,ecad2386,1


In [9]:
itemIdCategorySize=len(questionDf["item_id"].value_counts())
userIdCategorySize=len(questionDf["user_id"].value_counts())

In [10]:
latent_factor=10
regularizationFactor=0.00001
learningRate=0.0001
neuronSize=32
itemIdEmbeddingVectorL1 = Input(shape=[1])
itemIdEmbeddingVectorL2 = Embedding(itemIdCategorySize, latent_factor)(itemIdEmbeddingVectorL1)

In [11]:
itemIdEmbeddingVectorL3=Reshape(target_shape=(latent_factor,))(itemIdEmbeddingVectorL2)

In [12]:
userIdEmbeddingVectorL1 = Input(shape=[1])
userIdEmbeddingVectorL2 = Embedding(userIdCategorySize, latent_factor)(userIdEmbeddingVectorL1)

In [13]:
userIdEmbeddingVectorL3=Reshape(target_shape=(latent_factor,))(userIdEmbeddingVectorL2)

In [20]:
combinedL2 = Concatenate(axis=1)([itemIdEmbeddingVectorL3, userIdEmbeddingVectorL3])
combinedL3 = Reshape(target_shape=(latent_factor*2,))(combinedL2) 
#combinedL3 = Flatten()(combinedL2) 
denseL1=Dense(neuronSize,kernel_initializer='glorot_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(regularizationFactor))(combinedL3)
denseL2=Dense(1, activation='sigmoid')(denseL1)

In [21]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learningRate, beta_1=0.99, beta_2=0.999)
MLP = Model(inputs=[itemIdEmbeddingVectorL1, userIdEmbeddingVectorL1], outputs=[denseL2])
MLP.compile(optimizer=optimizer , loss='binary_crossentropy', metrics=[keras.metrics.AUC()])

In [22]:
MLP.summary()


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 10)        23950       input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1, 10)        5000000     input_2[0][0]                    
____________________________________________________________________________________________

In [23]:
#itemData=np.array(questionDf["item_id"].astype(str))
#userData=np.array(questionDf["user_id"])

In [24]:
#from keras.preprocessing.text import one_hot
#itemDataOneHot = [one_hot(d, itemIdCategorySize) for d in itemData]
#userDataOneHot = [one_hot(d, userIdCategorySize) for d in userData]
users = questionDf.user_id.unique()
userid2idx = {o:i for i,o in enumerate(users)}
usersDataOneHot= questionDf["user_id"].apply(lambda x: userid2idx[x])

items = questionDf.item_id.unique()
itemsid2idx = {o:i for i,o in enumerate(items)}
itemsDataOneHot= questionDf["item_id"].apply(lambda x: itemsid2idx[x])

usersDataOneHot=np.array(usersDataOneHot)
itemsDataOneHot=np.array(itemsDataOneHot)
target=np.array(questionDf["target"])

In [25]:
stop_noimprovement = keras.callbacks.EarlyStopping(patience=5)
filepath="1stQuestionMLP.hdf5"
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss',
                                                   save_best_only=True, mode='min')
epochSize=20
batchSize=32
MLP.fit([itemsDataOneHot,usersDataOneHot], target, batch_size=batchSize,
        epochs=epochSize, validation_split=0.2, verbose=2,
        callbacks=[stop_noimprovement,model_checkpoint])

Train on 400000 samples, validate on 100000 samples
Epoch 1/20
 - 1138s - loss: 0.6608 - auc_2: 0.6040 - val_loss: 0.6564 - val_auc_2: 0.6064


c:\users\casper\appdata\local\programs\python\python37\lib\site-packages\keras\engine\saving.py:165: UserWarning: TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file.You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
  'TensorFlow optimizers do not '


Epoch 2/20
 - 1411s - loss: 0.6369 - auc_2: 0.6829 - val_loss: 0.6678 - val_auc_2: 0.6029
Epoch 3/20
 - 1406s - loss: 0.5374 - auc_2: 0.8083 - val_loss: 0.7665 - val_auc_2: 0.5769
Epoch 4/20
 - 1378s - loss: 0.3860 - auc_2: 0.9089 - val_loss: 0.9281 - val_auc_2: 0.5563
Epoch 5/20
 - 1366s - loss: 0.2449 - auc_2: 0.9647 - val_loss: 1.1212 - val_auc_2: 0.5414
Epoch 6/20
 - 1368s - loss: 0.1379 - auc_2: 0.9890 - val_loss: 1.3203 - val_auc_2: 0.5318


In [ ]:
model = keras.models.load_model(modelName)

In [ ]:
#predictFuture = model.predict(predictSplit)